In [41]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import matplotlib.pyplot as plt
import math

# kNN classifier code

In [ ]:
#Importing the required modules
import numpy as np
from scipy.stats import mode
 
#Euclidean Distance
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
 
#Function to calculate KNN
def predict(x_train, y , x_input, k):
    op_labels = []
    #Loop through the Datapoints to be classified
    for item in x_input: 
        #Array to store distances
        #point_dist = []
        point_dist=np.zeros(x_input.shape[0])
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item)#distance.euclidean()# 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
        #Sorting the array while preserving the index, and keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
        #Labels of the K datapoints from above
        labels = y[dist]
        #Majority voting using mode()
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)
 
    return op_labels

# Feature Extraction

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
vggmodel = hub.load('https://tfhub.dev/google/vggish/1')

def embedding_from_fn(fn):
    x, sr = librosa.load(fn,sr=None)
    x_16k = librosa.resample(x,sr,16000) #resample to 16KHz
    embedding = np.array(vggmodel(x_16k))
    return embedding

In [42]:
import numpy as np
import scipy.signal
import scipy.io.wavfile

fs,x = scipy.io.wavfile.read('./groove/drummer1/session1/3_funk_80_beat_4-4.wav')
audio=[]
for i in range(x.shape[0]):
    audio.append((x[i][0]+x[i][1])/2)
audio=np.array(audio)
audio = audio/max(audio)

<ipython-input-42-6b445b4c9d8c>:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs,x = scipy.io.wavfile.read('./groove/drummer1/session1/3_funk_80_beat_4-4.wav')
<ipython-input-42-6b445b4c9d8c>:8: RuntimeWarning: overflow encountered in short_scalars
  audio.append((x[i][0]+x[i][1])/2)


In [22]:
embedding = np.array(vggmodel(audio))

In [40]:
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    # compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs   
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]    
    return (xb,t)

In [44]:
def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def mel_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)

def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, fs=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = mel_to_freq(mels)
    
    return np.floor((FFT_size + 1) / fs * freqs).astype(int), freqs

def get_filters(filter_points, FFT_size):
    filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
    return filters
    
def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num,filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)
    
    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
    return basis

In [70]:
xb,t = block_audio(audio,2048,1024,fs)

In [75]:
n_mfccs=13

freq_min = 0
freq_high = fs / 2
fft_size=xb[0].shape[0]
dct_filter_num = 40
dct_filters = dct(dct_filter_num, n_mfccs)
window = get_window("hann",fft_size , fftbins=True)

audio_winT = np.transpose(xb*window)

audio_fft = np.empty((int(1 + fft_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

for n in range(audio_fft.shape[1]):
    audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

audio_fft = np.transpose(audio_fft)
audio_power = np.square(np.abs(audio_fft))
filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
filters = get_filters(filter_points, fft_size)

enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
filters *= enorm[:, np.newaxis]

audio_filtered = np.dot(filters, np.transpose(audio_power))
audio_log = 10.0 * np.log10(audio_filtered)

dct_filters = dct(dct_filter_num, n_mfccs)

cepstral_coefficents = scipy.fftpack.dct(audio_log)#np.dot(dct_filters, audio_log)


<ipython-input-75-42337085eaf8>:26: RuntimeWarning: divide by zero encountered in log10
  audio_log = 10.0 * np.log10(audio_filtered)


In [79]:
audio_fft.shape

(5294, 1025)

In [81]:
cepstral_coefficents

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [66]:
def calc_mfccs(xb,fs=44100,n_mfccs=13):
    freq_min = 0
    freq_high = fs / 2
    fft_size=xb[0].shape[0]
    dct_filter_num = 40
    dct_filters = dct(dct_filter_num, n_mfccs)
    window = get_window("hann",fft_size , fftbins=True)
    mfccs=[]
    for block in xb:
        # Apply Window to the block
        windowed_block = window * block
        stft_blk = np.fft.fft(windowed_block)[:windowed_block.shape[0]]
        #stft_blk = np.fft.rfft(windowed_block)
        stft_blk = np.square(np.abs(stft_blk)) #POWER SPECTRUM NOT MAGNITUDE
        #stft_block = stft_blk[int((stft_blk.shape[0])/2):]
        filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
        filters = get_filters(filter_points, fft_size)
        enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
        filters *= enorm[:, np.newaxis]
        audio_filtered = np.dot(filters, np.transpose(stft_blk))
        audio_log = 10.0 * np.log10(audio_filtered)
        cepstral_coefficents = np.dot(dct_filters, audio_log)
        mfccs.append(cepstral_coefficents)
    return mfccs

In [69]:
window = get_window("hann",fft_size , fftbins=True)
windowed_block = window * block
stft_blk = np.fft.fft(windowed_block)
#stft_blk = np.fft.rfft(windowed_block)
stft_blk = np.square(np.abs(stft_blk)) #POWER SPECTRUM NOT MAGNITUDE
#stft_block = stft_blk[int((stft_blk.shape[0])/2):]
filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
filters = get_filters(filter_points, fft_size)
enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
filters *= enorm[:, np.newaxis]
audio_filtered = np.dot(filters, np.transpose(stft_blk))
audio_log = 10.0 * np.log10(audio_filtered)
cepstral_coefficents = np.dot(dct_filters, audio_log)

ValueError: shapes (13,1025) and (2048,5294) not aligned: 1025 (dim 1) != 2048 (dim 0)

In [47]:

calc_mfccs(xb,44100,13)


ValueError: shapes (13,1025) and (2048,) not aligned: 1025 (dim 1) != 2048 (dim 0)

In [ ]:
def agg_mfccs(x):
    mfccs = librosa.feature.mfcc(x,n_mfcc = 13)
    mean = [np.mean(feature) for feature in mfccs]
    std = [np.std(feature) for feature in mfccs]
    mfcc_delta = librosa.feature.delta(mfccs)
    delta_mean=[np.mean(feature) for feature in mfcc_delta]
    delta_std=[np.std(feature) for feature in mfcc_delta]
    return mean,std,delta_mean,delta_std